**1. INTRODUCTION**

Toronto is the provincial city of Ontario and the most populous city in Canada. It had a population of 2.731.571 people in 2016. This city has too many places to visit, like the Royal Ontario Museum, the Art Gallery of Ontario, the Gardiner museum of Ceramic Art and Ontario Science Centre (Wikipedia, 2019).
Toronto in an international centre for business and finance. It has a high concentration of banks and brokerage firms on Bay street. The city is an important centre for the media. Some of the corporations are the Bell media, Rogers communication and Torstar (Wikipedia, 2019).


**2. Objective**

Marcos is a cooker that lives in the borough of Etobicoke and at the neighborhood of Cloverdale in Toronto. He comes from a family of famous cookers of Canada. He wants to build his first restaurant at Canada but he wants to build in a place that has the same characteristics as his neighborhood. He could build his first restaurant at this neighborhood, but he wants to know if there are other places with the same characteristics. He looked for a statistician to see if there are other places in Toronto with the same characteristics as his neighborhood. So, let's get the job done and find an answer for Marcos.

**3. Dataset**

The dataset used at this project come from different sources. One of the datasets is from wikipedia dataset that contains the boroughs and neighborhoods from the city of Toronto (link: <https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M>), and the other is from foursquare. It will be used to generate the geolocalization for the neighborhoods and its respective venues.

The datasets are gonna be used to generate a cluster model to find the neighborhoods that are similar to Cloverdale, according to the objective of this project.